In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
url = "https://www.ccny.cuny.edu/registrar/fall"
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)
# print(soup.prettify())

In [3]:
table = soup.find_all('table')[0]
# print(table)

rows = table.find_all('tr')
# print(rows)

In [4]:
calendar_rows = {
    "date": [],
    "day of the week": [],
    "text": [],
}


# Returns the start date explicitly
def clean_date(date):
    default_year = ", 2021"

    start_date = date.strip().split("-")[0]

    if len(start_date.split(",")) == 1:
        start_date = start_date.strip() + default_year

    return datetime.strptime(start_date, "%B %d, %Y").date()

# Returns the day of the week mentioned
def clean_dotw(dotw):
    return dotw.strip().replace("\n", "").replace("\t", "").split(" ")[0]


def clean_text(text):
    return text.strip().replace("\n", " ").replace("\t", "")


for row in rows:
    row_data = row.find_all("td")

    if row_data:
        calendar_rows["date"].append(clean_date(row_data[0].text))
        calendar_rows["day of the week"].append(clean_dotw(row_data[1].text))
        calendar_rows["text"].append(clean_text(row_data[2].text))

# print(calendar_rows)

In [5]:
# Converting dictionary to pandas dataframe
df = pd.DataFrame(calendar_rows)
df = df.set_index("date")

print(df)

           day of the week                                               text
date                                                                         
2021-08-01          Sunday  Application for degree for January and Februar...
2021-08-18       Wednesday                 Last day to apply for Study Abroad
2021-08-24         Tuesday  Last day of Registration; Last day to file ePe...
2021-08-25       Wednesday  Start of Fall Term; Classes begin; Initial Reg...
2021-08-25       Wednesday          Change of program period; late fees apply
2021-08-26        Thursday                     Last day for Independent Study
2021-08-28        Saturday                      First day of Saturday Classes
2021-08-31         Tuesday  Last day to add a class to an existing enrollm...
2021-09-01       Wednesday  Verification of Enrollment rosters available t...
2021-09-03          Friday                               No classes scheduled
2021-09-06          Monday   College Closed; Last day for 50% tu